In [110]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score
from joblib import dump, load

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extract data samples from raw GDF files

In [111]:
%%capture
convert_data()


# Read and split the data

In [112]:
i = 7

x_train, y_train = read_file("action_train", f"A0{i}")
x_test, y_test = read_file("action_test", f"A0{i}")


print("final shapes are: ", x_train.shape, y_train.shape)
print("final shapes are: ", x_test.shape, y_test.shape)


final shapes are:  (230, 25, 750) (230,)
final shapes are:  (58, 25, 750) (58,)


# Extract features from test and train data

In [113]:
%%capture
train_coeff = featurize(x_train)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
x_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)

test_coeff = featurize(x_test)
x_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)
  

# Train model and test

In [114]:
clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
clf.fit(x_train_f, y_train)

print("Accuracy is ", sum(clf.predict(x_train_f) == y_train) / len(x_train))
print(len(x_train))

print("test Accuracy is ", sum(clf.predict(x_test_f) == y_test) / len(x_test))
print(len(x_test))

print("kappa score on train is: ", cohen_kappa_score(clf.predict(x_train_f), y_train))
print("kappa score on test is: ", cohen_kappa_score(clf.predict(x_test_f), y_test))


Accuracy is  1.0
230
test Accuracy is  0.9137931034482759
58
kappa score on train is:  1.0
kappa score on test is:  0.884737678855326


# Save the trained model

In [115]:
dump(clf, "model.joblib")
dump(csp, "csp.joblib")

['csp.joblib']